# extract audio feature

## 01 test sample

In [1]:
# import os
# import csv

In [2]:
# path = '../data/raw/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro01/'
# file_name = path + 'Ses01F_impro01_F002.wav'

In [3]:
# out_file = '../data/processed/IEMOCAP/test.csv'

In [4]:
# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_prosodyShsViterbiLoudness.conf -I ' + file_name + ' -O ' + out_file + ' -headercsv 1'

# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A_Z.conf -I ' + file_name + ' -O ' + out_file  + ' -headercsv 1'
# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A.conf -I ' + file_name + ' -csvoutput ' + out_file  + ' -headercsv 0' # MFCC12EDA case

# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_emobase2010.conf -I ' + file_name + ' -csvoutput ' + out_file  + ' -headercsv 1'

In [5]:
# print cmd
# os.system(cmd)

In [6]:
# lines = []
# with open('../data/processed/IEMOCAP/processeed_prosody.csv') as f:
#     csv_reader = csv.reader(f)
#     lines = [x for x in csv_reader]        

## 01 extract MFCC / prosody / emobase feature from multiple files

In [4]:
import os
import sys
import csv
from file_util import *

### select opensmile configuration

### 01-1 MFCC EDA

In [11]:
# MFCC E D A Z
# OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A_Z.conf'   # normalized 
# out_file = '../data/processed/IEMOCAP/MFCC12EDAZ.csv'
    
    
# MFCC E D A    
OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A.conf'    
out_file = '../data/processed/IEMOCAP/MFCC12EDA.csv'
    
out_file    

'../data/processed/IEMOCAP/MFCC12EDA.csv'

In [12]:
def extract_feature( list_in_file, out_file ) :
    print out_file
    
    cnt = 0    
    for in_file in list_in_file:        
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -O ' + out_file + ' -headercsv 0'  #MFCC12EDAZ, prosody
        cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # MFCC12EDA
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # emobase2010
        os.system(cmd)
        
        cnt += 1
        if cnt % 1000 == 0:
            print str(cnt) + " / " + str( len(list_in_file) )
            sys.stdout.flush()

In [13]:
# [schema] ID, transcriptions [csv]

list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))

extract_feature( list_files, out_file )

Session1, #sum files: 1819
Session2, #sum files: 3630
Session3, #sum files: 5766
Session4, #sum files: 7869
Session5, #sum files: 10039
../data/processed/IEMOCAP/MFCC12EDA.csv
1000 / 10039
2000 / 10039
3000 / 10039
4000 / 10039
5000 / 10039
6000 / 10039
7000 / 10039
8000 / 10039
9000 / 10039
10000 / 10039


In [19]:
with open('../data/processed/IEMOCAP/MFCC12EDA.csv') as f:
    data = f.readlines()
len(data)

4457391

In [20]:
data[0]

"'unknown';0.000000;-6.109069e+00;-1.402484e+01;6.732327e+00;-2.546580e+01;6.154059e+00;-1.788557e+01;7.608350e+00;-2.189162e+01;-2.357424e+00;3.110782e+00;1.537108e+01;2.076875e+00;1.549505e+01;9.102157e-01;-1.072150e+00;1.679795e+00;1.134924e+00;-1.113729e+00;-1.536331e+00;-3.516938e-01;2.706552e+00;2.835383e+00;-4.857984e-01;-2.045343e+00;-4.918585e+00;1.934452e-02;1.112185e-01;4.612769e-01;-4.294967e-01;2.867506e-01;4.323770e-01;7.239794e-01;8.828823e-01;6.933632e-02;1.803990e-01;-5.409565e-01;-2.570946e-01;4.309415e-01;-2.529488e-02\n"

### 01-2 prosody

In [1]:
# prosody
OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_prosodyShsViterbiLoudness.conf'
out_file = '../data/processed/IEMOCAP/prosody.csv'
    
out_file    

'../data/processed/IEMOCAP/prosody.csv'

In [5]:
def extract_feature( list_in_file, out_file ) :
    print out_file
    
    cnt = 0    
    for in_file in list_in_file:        
        cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -O ' + out_file + ' -headercsv 0'  #MFCC12EDAZ, prosody
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # MFCC12EDA
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # emobase2010
        os.system(cmd)
        
        cnt += 1
        if cnt % 1000 == 0:
            print str(cnt) + " / " + str( len(list_in_file) )
            sys.stdout.flush()

In [6]:
# [schema] ID, transcriptions [csv]

list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))

extract_feature( list_files, out_file )

Session1, #sum files: 1819
Session2, #sum files: 3630
Session3, #sum files: 5766
Session4, #sum files: 7869
Session5, #sum files: 10039
../data/processed/IEMOCAP/prosody.csv
1000 / 10039
2000 / 10039
3000 / 10039
4000 / 10039
5000 / 10039
6000 / 10039
7000 / 10039
8000 / 10039
9000 / 10039
10000 / 10039


### 01-3 emobase

In [7]:
# emobase2010
OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_emobase2010.conf'    
out_file = '../data/processed/IEMOCAP/emobase2010.csv'
    
out_file    

'../data/processed/IEMOCAP/emobase2010.csv'

In [ ]:
def extract_feature( list_in_file, out_file ) :
    print out_file
    
    cnt = 0    
    for in_file in list_in_file:        
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -O ' + out_file + ' -headercsv 0'  #MFCC12EDAZ, prosody
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # MFCC12EDA
        cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # emobase2010
        os.system(cmd)
        
        cnt += 1
        if cnt % 1000 == 0:
            print str(cnt) + " / " + str( len(list_in_file) )
            sys.stdout.flush()

In [ ]:
# [schema] ID, transcriptions [csv]

list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))

extract_feature( list_files, out_file )

Session1, #sum files: 1819
Session2, #sum files: 3630
Session3, #sum files: 5766
Session4, #sum files: 7869
Session5, #sum files: 10039
../data/processed/IEMOCAP/emobase2010.csv
1000 / 10039


## 02 verify

In [49]:
import csv
from file_util import *

In [50]:
in_file = '../data/processed/IEMOCAP/label.csv'
label = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    label = [x for x in csv_reader]
    
label_id = [ x[0] for x in label]

In [51]:
list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print sess_name + ", #sum files: " + str(len(list_files))
    
ids = [ x.split('/')[-1].split('.')[0] for x in list_files ]

Session1, #sum files: 1819
Session2, #sum files: 3630
Session3, #sum files: 5766
Session4, #sum files: 7869
Session5, #sum files: 10039


In [52]:
for l, t in zip(label_id, ids):
    if l != t:
        print 'ERROR'

## 03 A MFCC post-process
- each row has MFCC feature information (41 dimesion) * audio length --> can be different according to its length
- [0]frameIndex; [1]frameTime; were removed from processed feature
- (10039, 750, 39)

In [15]:
import csv
import cPickle
import numpy as np

#### 03-A load MFCC feature

In [16]:
lines = []
# with open('../data/processed/IEMOCAP/MFCC12EDAZ.csv') as f:
with open('../data/processed/IEMOCAP/MFCC12EDA.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

In [21]:
len(lines)

4457391

#### 03-B MFCC feature -> float value

In [22]:
lines = [x[0] for x in lines]

In [23]:
lines = [x.split(';') for x in lines]

In [24]:
float_lines = [ [float(i) for i in x[1:]] for x in lines ]   # do not care the first element

In [25]:
# mark the index of each chunk 

chunk_index = []
for i, line in enumerate(float_lines):
    if line[0] == 0:
        chunk_index.append(i)

In [26]:
no_index_float_linex = [ x[2:] for x in lines ]   # remove first two element (sequence index)

In [27]:
# merge sequence 

list_MFCC = []

for i in xrange( len(chunk_index) ):
    
    if i == len(chunk_index) -1:        # last case
        list_MFCC.append( no_index_float_linex[ chunk_index[i]: ] )
    else :
        list_MFCC.append( no_index_float_linex[ chunk_index[i]:chunk_index[i+1] ] )

#### 03-C numpy save  7.5s cut

In [28]:
stat = [ len(x) for x in list_MFCC ]
print np.mean(stat)
print np.std(stat)
print np.max(stat)
print np.min(stat)

444.00747086363185
306.4563457098706
3412
56


In [29]:
# np.save('../data/processed/IEMOCAP/processed_MFCC12EDAZ_sequenceN.npy', np.asarray(stat))
np.save('../data/processed/IEMOCAP/processed_MFCC12EDA_sequenceN.npy', np.asarray(stat))

In [30]:
# with open('../data/processed/IEMOCAP/processed_MFCC12EDAZ_sequenceN.txt', 'w') as f:
with open('../data/processed/IEMOCAP/processed_MFCC12EDA_sequenceN.txt', 'w') as f:
    for data in stat:
        f.write( str(data) + '\n' )

In [31]:
np_MFCC = np.zeros( [10039, 750, 39], dtype=np.float)
np.shape(np_MFCC)

(10039, 750, 39)

In [32]:
for i in xrange( len(list_MFCC) ):
    
    if len( list_MFCC[i] ) > 750:
        np_MFCC[i][:] = list_MFCC[i][:750]
    else:
        np_MFCC[i][:len(list_MFCC[i])] = list_MFCC[i][:]

In [33]:
# np.save('../data/processed/IEMOCAP/processed_MFCC12EDAZ.npy', np_MFCC)
np.save('../data/processed/IEMOCAP/processed_MFCC12EDA.npy', np_MFCC)

## 04 prosody post-process
- [0]@name string; [36]class numeric; were removed from original feature (total 37)
- (10039, 35) 

#### 04-A load prosody feature  &  conver to np format

In [39]:
lines = []
with open('../data/processed/IEMOCAP/prosody.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]
len(lines)

10039

In [40]:
np_prosody = np.zeros( [10039, 35], dtype=np.float)
np.shape(np_prosody)

(10039, 35)

In [41]:
for i in xrange( len(np_prosody) ):
    np_prosody[i] = lines[i][1:-1]    

#### 04-B  numpy save

In [42]:
np.save('../data/processed/IEMOCAP/processed_prosody.npy', np_prosody)

## 05 emobase2010 post-process
- [0]frameTime aws removed from original feature (total 1583)
- (10039, 1582) 

#### 05-A load emobase2010 feature  &  conver to np format

In [48]:
lines = []
with open('../data/processed/IEMOCAP/emobase2010.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

lines = [x[0].split(';') for x in lines]
lines = [ [float(t) for t in x] for x in lines]
len(lines)

10039

In [44]:
np_emobase = np.zeros( [10039, 1582], dtype=np.float)
np.shape(np_emobase)

(10039, 1582)

In [45]:
for i in xrange( len(np_emobase) ):
    np_emobase[i] = lines[i][1:]    

#### 04-B  numpy save

In [46]:
np.save('../data/processed/IEMOCAP/processed_emobase2010.npy', np_emobase)

In [47]:
tmp = np.load( '../data/processed/IEMOCAP/processed_emobase2010.npy')
np.shape(tmp)

(10039, 1582)